In [ ]:
import cv2
import numpy as np

cap = cv2.VideoCapture(0)
print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

#Create old frame
_,frame = cap.read()
old_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

#Lucas Kanade params
lk_params = dict(winSize = (25,25),#search
                maxLevel = 2,
                criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#mouse function
def select_point(event, x, y, flags, params):
    global point, point_selected, old_points
    if event == cv2.EVENT_LBUTTONDOWN:
        point = (x, y)
        point_selected = True
        old_points = np.array([[x,y]], dtype=np.float32)

cv2.namedWindow("Frame", cv2.WINDOW_NORMAL)
cv2.setMouseCallback("Frame",select_point)
point_selected = False
point = ()
#old_points = np.array([[]])
#old_points =np.array([[[312.,220.]],[[400.,220.]],[[461.,220.]]],dtype=np.float32)
old_points =np.array([[[200.,220.]]],dtype=np.float32)
count = 0

while True:
    ret, frame = cap.read()
    
    if not ret:
        break

    if count < 50:
        mask = np.zeros_like(frame)
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    for i in range(len(old_points)):
        x, y = old_points[i].ravel()
        #if point_selected is True:
        cv2.circle(frame, (int(x), int(y)), 5, (0,0,255),2)

    new_points, status, error = cv2.calcOpticalFlowPyrLK(old_gray, gray_frame, 
                                                         old_points, None, **lk_params)
    
    
    good_new = new_points[status==1]
    good_prev = old_points[status==1]
    
    for i,(new,prev) in enumerate(zip(good_new,good_prev)):
        x_new,y_new = new.ravel()
        x_prev,y_prev = prev.ravel()    
        #cv2.line(frame, (x_new,y_new),(x_prev,y_prev), (0,255,0), 1)
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), (0,255,0), 2)
        #frame = cv2.circle(frame,(x_new,y_new),4,(0,0,255),-1)
    
    old_gray = gray_frame.copy()
    #print(old_points)
    #print(np.shape(old_points))
    old_points = new_points
    
    for i in range(len(new_points)):
        x, y = new_points[i].ravel()
        #if point_selected is True:
        cv2.circle(frame, (int(x), int(y)), 5, (0,255,0), -1)
    
    #first_level = cv2.pyrDown(frame)
    #second_level = cv2.pyrDown(first_level)
    img = cv2.add(frame,mask)
    
    cv2.imshow("Frame",img)
    #cv2.imshow("Frame level", first_level)
    #cv2.imshow("Second level", second_level)
    
    if cv2.waitKey(30) & 0xFF == ord('q'):
        break

    count +=1

cap.release()
cv2.destroyAllWindows()


640.0
480.0
